In [2]:
import datetime
import scipy.stats as stats
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro
matplotlib.use('Agg')
%matplotlib inline
from dateutil.relativedelta import relativedelta

from finrl import config
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline, get_baseline_tdx,convert_daily_return_to_pyfolio_ts

import torch
# import plotly.express as px

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import EfficientFrontier
from pypfopt import expected_returns
from pypfopt import objective_functions
from pyfolio import timeseries

import plotly.graph_objects as go
from plotly.subplots import make_subplots

d:\code\python\pyfolio\pyfolio\pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [3]:
from lutils.stock import LTdxHq

In [4]:
train_start = '2019-01-01'
train_end = '2022-07-01'
trade_start = train_end
trade_end = '2022-07-12'

In [5]:
stock_count = 4

In [6]:
ddf = pd.read_pickle('d:/ddf.pkl').dropna()
# ddf = pd.read_pickle('d:/ddf_0606.pkl').dropna()

In [7]:
ddf = ddf.sort_values(['date','tic'], ignore_index=True)
ddf.index = ddf.date.factorize()[0]

In [8]:
days = ddf.date.unique()
days.shape

(1495,)

In [9]:
dl = ddf[(ddf['date'] > days[-63]) & (ddf['date'] < trade_start)]
dd = dl.pivot_table(index = 'date',columns = 'tic', values = 'close').pct_change().dropna()
corr = dd.cov().corr()
# corr.style.background_gradient(cmap='coolwarm')

# c1 = corr.abs().unstack().sort_values(ascending = True)
# c1 = corr.unstack().sort_values(ascending = True)

In [10]:
# corr.style.background_gradient(cmap='coolwarm')

In [11]:
# corr.unstack().sort_values().index.get_level_values(0)[:10]

In [12]:
corr.sum().sort_values() # ascending=False

tic
300435   -2611.0
002920   -2611.0
600968   -2611.0
002890   -2611.0
601699   -2611.0
           ...  
688555    2611.0
002315    2611.0
000949    2611.0
002378    2611.0
300106    2611.0
Length: 3547, dtype: float64

In [13]:
# stock_codes = list(np.random.choice(ddf.tic.unique(), 6))

In [14]:
stock_codes = list(corr.unstack().sort_values().index.get_level_values(0)[:stock_count])
# ['601398', '000738']
# ['601988', '603538'] # 2022-02-08
# ['600377', '000887'] # 2022-02-14
# ['301040', '000573'] # 2022-02-15
# ['301040', '000573', '300384'] # 2022-02-20

In [15]:
# stock_codes = list(corr.sum().sort_values().index.values[:2]) # ascending=False
# ['000921', '600365', '000609', '601868', '002372', '001965']
# ['601868', '000501', '603789', '002311', '002873', '000921']
# ['000921', '603789', '002372', '601868', '300675', '002032']
# ['600775', '603789', '300622', '603228', '600172', '001965']

In [16]:
# stock_codes = ['601186', '002645', '600546', '688222', '600107', '603028']
# stock_codes = ['601988', '603538']

In [17]:
stock_codes

['603003', '603326', '002533', '600459']

In [18]:
# stock_codes = ['000921', '002032', '300406', '603789']
# stock_codes = ['000609', '000921', '001965', '002372']

In [19]:
ltdxhq = LTdxHq()

indexs = None
dfs = []
for code in stock_codes:
    df = ltdxhq.get_k_data_daily(code, start='2018-01-01') # 2014-01-01
    
    if indexs is None:
        indexs = df.index
    else:
        indexs = indexs.union(df.index)
    
#     df = df.assign(date = df.index)
#     df = df.assign(day = df.index.weekday)
#     df.date = df.date.dt.strftime('%Y-%m-%d')
    df = df.assign(tic = code)
#     df.index = range(df.shape[0])
    
    dfs.append(df)
    print('----------- over %s min: %s max: %s -----------' % (code, df.index.min(), df.index.max()))

for i, df in enumerate(dfs):
    df = df.reindex(indexs)
    df = df.assign(date = df.index)
    df = df.assign(day = df.index.weekday)
    df.index = range(df.shape[0])
    
    dfs[i] = df.ffill()
    
# df = pd.concat(dfs)
# df.index = range(df.shape[0])

ltdxhq.close()

----------- over 603003 min: 2018-01-02 00:00:00 max: 2022-07-11 00:00:00 -----------
----------- over 603326 min: 2018-01-02 00:00:00 max: 2022-07-11 00:00:00 -----------
----------- over 002533 min: 2018-01-02 00:00:00 max: 2022-07-11 00:00:00 -----------
----------- over 600459 min: 2018-01-02 00:00:00 max: 2022-07-11 00:00:00 -----------


In [20]:
df = pd.concat(dfs)

In [21]:
%matplotlib inline

In [22]:
# hist = df[df.tic == '601868 '].close.hist() # bins=3)

# df[['open', 'close', 'high', 'low']].plot.hist(bins=100, alpha=0.5)
# df[df.tic == '000921'].close.pct_change().hist(bins=100)
# sns.distplot(df[df.tic == '000921'].close)

In [23]:
# sns.distplot(df[df.tic == '000921'].close.pct_change())

In [24]:
# stat, p = shapiro(df[df.tic == '000921'].close.pct_change())
# stat, p

In [25]:
fe = FeatureEngineer(use_technical_indicator=True,
                     use_turbulence=False,
                     user_defined_feature = False)

df = fe.preprocess_data(df)

Successfully added technical indicators


In [26]:
# add covariance matrix as states
df=df.sort_values(['date','tic'], ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
return_list = []

# look back is one year
lookback=252
for i in range(lookback, len(df.index.unique())):
    data_lookback = df.loc[i-lookback:i, :]
    price_lookback = data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
    return_lookback = price_lookback.pct_change().dropna()
    return_list.append(return_lookback)

    covs = return_lookback.cov().values
    cov_list.append(covs)


df_cov = pd.DataFrame({'date': df.date.unique()[lookback:], 'cov_list': cov_list,'return_list': return_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date', 'tic']).reset_index(drop = True)

In [27]:
df['daily_variance'] = (df.high - df.low) / df.close

In [28]:
dl = df.loc[-5:, :]
dd = dl.pivot_table(index = 'date',columns = 'tic', values = 'close').pct_change().dropna()
dd.cov().corr().style.background_gradient(cmap='coolwarm')

tic,002533,600459,603003,603326
tic,,,,
002533,1.000000,-0.054450,-0.340061,-0.598607
600459,-0.054450,1.000000,-0.169959,-0.539503
603003,-0.340061,-0.169959,1.000000,-0.204035
603326,-0.598607,-0.539503,-0.204035,1.000000


In [29]:
train = data_split(df, train_start, train_end) # 2021-07-01 2022-01-01

In [30]:
train.head()

,open,close,high,low,volume,amount,tic,date,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list,daily_variance
0,4.07,4.10,4.11,4.04,3144799.0,15198757.0,002533,2019-01-15,1,0.052644,4.187461,3.603539,55.000143,89.313573,31.599010,3.930000,3.863167,"[[0.00042213065949817574, 0.000349382372041037...",tic 002533 600459 603003 60...,0.017073
0,9.45,9.36,9.53,9.26,18742920.0,254009680.0,600459,2019-01-15,1,0.284946,9.391885,7.315115,60.981093,201.533014,51.896866,8.396000,7.958333,"[[0.00042213065949817574, 0.000349382372041037...",tic 002533 600459 603003 60...,0.028846
0,7.11,7.15,7.16,7.07,1928959.0,13743916.0,603003,2019-01-15,1,-0.040264,7.312258,6.622742,48.118470,4.533813,11.849026,7.115000,7.174667,"[[0.00042213065949817574, 0.000349382372041037...",tic 002533 600459 603003 60...,0.012587
0,8.49,9.03,9.11,8.43,6221121.0,81031064.0,603326,2019-01-15,1,0.379620,8.733203,6.767797,73.506136,281.748564,43.470020,7.626667,7.113500,"[[0.00042213065949817574, 0.000349382372041037...",tic 002533 600459 603003 60...,0.075305
1,4.10,4.05,4.13,4.05,2488900.0,11982817.0,002533,2019-01-16,2,0.053268,4.202526,3.602474,53.538617,86.274510,33.569463,3.928000,3.869667,"[[0.0004225857775121337, 0.000349646215916637,...",tic 002533 600459 603003 60...,0.019753


In [31]:
train[train['cci_30'] == np.inf]

,open,close,high,low,volume,amount,tic,date,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list,daily_variance


In [32]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv


class StockPortfolioEnv(gym.Env):
    """A portfolio allocation environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then 
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step
        

    """
    metadata = {'render.modes': ['human']}

    def __init__(self,
                 df,
                 stock_dim,
                 hmax,
                 initial_amount,
                 transaction_cost_pct,
                 reward_scaling,
                 state_space,
                 action_space,
                 tech_indicator_list,
                 turbulence_threshold=None,
                 lookback=252,
                 day=0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
#         self.reward = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low=0, high=1,shape=(self.action_space,))
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False
        self.turbulence_threshold = turbulence_threshold
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1 / self.stock_dim] * self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

        
    def step(self, actions):
        self.terminal = self.day >= len(self.df.index.unique()) - 1

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
#             plt.plot(df.daily_return.cumsum(),'r')
#             plt.savefig('results/cumulative_reward.png')
#             plt.close()
            
#             plt.plot(self.portfolio_return_memory,'r')
#             plt.savefig('results/rewards.png')
#             plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() != 0:
                sharpe = (252 ** 0.5) * df_daily_return['daily_return'].mean() / df_daily_return['daily_return'].std()
                print("Sharpe: ", sharpe)
            print("=================================")
            
            return self.state, self.reward, self.terminal,{}

        else:
            weights = self.softmax_normalization(actions)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state = np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list], axis=0)
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values) - 1) * weights)
#             log_portfolio_return = np.log(sum((self.data.close.values / last_day_memory.close.values) * weights))
            # update portfolio value
            new_portfolio_value = self.portfolio_value * (1 + portfolio_return)
#             new_portfolio_value = self.portfolio_value * (1 + log_portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value / self.initial_amount

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day, :]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]
        return self.state
    
    def render(self, mode='human'):
        return self.state
        
    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator / denominator
        return softmax_output

    
    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']
        
        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [33]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f'Stock Dimension: {stock_dimension}, State Space: {state_space}')

Stock Dimension: 4, State Space: 4


In [34]:
# ['daily_variance', 'change', 'log_volume', 'close','day', 'macd', 'rsi_30', 'boll_ub', 'dx_30']
# ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'boll_ub', 'dx_30', 'close_30_sma', 'close_60_sma'] # cci_30
tech_indicator_list = ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'daily_variance', 'dx_30', 'close_30_sma', 'close_60_sma']
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 10000 * stock_dimension, 
    "transaction_cost_pct": 0, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": tech_indicator_list, # config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-1
    
}

e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)

In [35]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [36]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    'n_steps': 2048,
    'ent_coef': 0.005,
    'learning_rate': 0.0001,
    'batch_size': 128,
}
model_ppo = agent.get_model('ppo', model_kwargs = PPO_PARAMS, tensorboard_log='D:/code/python/logs')

{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
Using cuda device


In [37]:
agent.train_model(model=model_ppo, tb_log_name='ppo', total_timesteps=20000)

d:\soft\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\soft\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\soft\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\soft\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

Logging to D:/code/python/logs\ppo_18
begin_total_asset:40000
end_total_asset:55982.22754080497
Sharpe:  0.4937659764277938
begin_total_asset:40000
end_total_asset:57545.48067385154
Sharpe:  0.5169323482631993
----------------------------------
| time/              |           |
|    fps             | 352       |
|    iterations      | 1         |
|    time_elapsed    | 5         |
|    total_timesteps | 2048      |
| train/             |           |
|    reward          | 1.4278675 |
----------------------------------
begin_total_asset:40000
end_total_asset:44292.18231079961
Sharpe:  0.2521725989068613
begin_total_asset:40000
end_total_asset:73492.68004690549
Sharpe:  0.7600356363899139
------------------------------------------
| time/                   |              |
|    fps                  | 407          |
|    iterations           | 2            |
|    time_elapsed         | 10           |
|    total_timesteps      | 4096         |
| train/                  |              |
| 

begin_total_asset:40000
end_total_asset:71599.88705685745
Sharpe:  0.7377019380764903
-----------------------------------------
| time/                   |             |
|    fps                  | 461         |
|    iterations           | 8           |
|    time_elapsed         | 35          |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.002821187 |
|    clip_fraction        | 0.00488     |
|    clip_range           | 0.2         |
|    entropy_loss         | -5.69       |
|    explained_variance   | -0.0151     |
|    learning_rate        | 0.0001      |
|    loss                 | 192         |
|    n_updates            | 70          |
|    policy_gradient_loss | -0.00526    |
|    reward               | 1.4657193   |
|    std                  | 1           |
|    value_loss           | 413         |
-----------------------------------------
begin_total_asset:40000
end_total_asset:61620.60689987623
Sharpe:  0.59061

In [38]:
trade = data_split(df, trade_start, trade_end) # '2021-12-20', '2023-01-01' trade_start, trade_end
e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)

In [39]:
unique_tic = trade.tic.unique()
unique_trade_date = trade.date.unique()

In [40]:
df_daily_return_ppo, df_actions_ppo = DRLAgent.DRL_prediction(model=model_ppo, environment=e_trade_gym)
time_ind = pd.Series(df_daily_return_ppo.date)
ppo_cumpod =(df_daily_return_ppo.daily_return + 1).cumprod() - 1
DRL_strat_ppo = convert_daily_return_to_pyfolio_ts(df_daily_return_ppo)

perf_func = timeseries.perf_stats 

perf_stats_all_ppo = perf_func(returns=DRL_strat_ppo, 
                               factor_returns=DRL_strat_ppo, 
                               positions=None, transactions=None, turnover_denom='AGB')

begin_total_asset:40000
end_total_asset:40268.246251865145
Sharpe:  1.2384835365213862
hit end!


In [38]:
# import matplotlib.pyplot as plt
# import squarify 
# %matplotlib inline
# sizes=[40, 30, 5, 25]
# label=["A", "B", "C", "D"]
# squarify.plot(sizes=sizes, label=label, alpha=0.6)
# plt.show()